In [ ]:
#!pip install --upgrade pip
#!pip install --upgrade setuptools
#!pip install GEOS
#!pip install GeobricksProj4ToEPSG
#!conda install -c conda-forge cartopy --yes
#!pip install geopandas
#!pip install msgpack
#!pip install geocoder
#print('ok')

In [ ]:
import os
import pandas
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML
import geopandas as gpd
import time
from shapely.geometry import Polygon, Point
from shapely.ops import cascaded_union
import random
import geocoder
from pyproj import Proj, transform

os.chdir('/home/idies/workspace/Storage/raddick/persistent/baltimore_neighborhoods/')
shapefile_dir = 'shapefiles/'

print(os.getcwd())

In [ ]:
s = time.time()
tracts_2010_filename = shapefile_dir + 'census_tracts_2010/geo_export_c50bbe56-543e-4878-9c9f-c56be327600a.shp'
tracts_2010_gdf = gpd.read_file(tracts_2010_filename, encoding='utf-8')
tracts_2010_gdf = tracts_2010_gdf.assign(tractname=pandas.to_numeric(tracts_2010_gdf['name'].apply(lambda x: x.split(' ')[-1]), errors='coerce'))
tracts_2010_gdf = tracts_2010_gdf.set_index('tractname')
#tracts_2010_gdf = tracts_2010_gdf.to_crs(neighborhood_gdf.crs)
e = time.time()
print('Got {0:,.0f} 2010 census tracts in {1:.3f} seconds.'.format(len(tracts_2010_gdf), e-s))

s = time.time()
tracts_2017_filename = shapefile_dir + 'cb_2017_24_bg_500k/cb_2017_24_bg_500k.shp'
tracts_2017_gdf = gpd.read_file(tracts_2017_filename, encoding='utf-8')
tracts_2017_gdf = tracts_2017_gdf.assign(tractname=pandas.to_numeric(tracts_2010_gdf['name'].apply(lambda x: x.split(' ')[-1]), errors='coerce'))
e = time.time()
print('Got {0:,.0f} 2017 census tracts in {1:.3f} seconds.'.format(len(tracts_2017_gdf), e-s))

tracts_2017_gdf = tracts_2017_gdf[tracts_2017_gdf['STATEFP'] == '24']
tracts_2017_gdf = tracts_2017_gdf[tracts_2017_gdf['COUNTYFP'] == '510']
print('Keeping {0:,.0f} 2017 census tracts in Baltimore City.'.format(len(tracts_2017_gdf)))

s = time.time()
census_tract_to_csa_filename = shapefile_dir + 'census_tract_to_neighborhood.csv'
census_tract_to_csa_df = pandas.read_csv(census_tract_to_csa_filename, low_memory=False)
census_tract_to_csa_df = census_tract_to_csa_df.set_index('NAME10')
tract_shapes_with_csa_df = tracts_2010_gdf.join(census_tract_to_csa_df, how='left')
csalist = tract_shapes_with_csa_df['CSA2010'].drop_duplicates().sort_values().tolist()
csa_df = pandas.DataFrame(data=csalist, columns=['CSA2010'])
csa_df = csa_df.set_index('CSA2010')
csa_df = csa_df.assign(geometry='')
for thiscsa, thisrow in csa_df.iterrows():
    tracts_here = tract_shapes_with_csa_df[tract_shapes_with_csa_df['CSA2010'] == thiscsa].index.values.tolist()
    geos_here = tract_shapes_with_csa_df[tract_shapes_with_csa_df.index.isin(tracts_here)].geometry
    csa_df.loc[thiscsa, 'geometry'] = geos_here.unary_union
csa_gdf = gpd.GeoDataFrame(csa_df, crs=tracts_2010_gdf.crs, geometry='geometry')
e = time.time()

print('Assembled 2010 census tracts into {0:,.0f} CSAs in {1:,.3f} seconds.'.format(len(csa_gdf), e-s))

s = time.time()
blockgroup_filename = shapefile_dir + 'cb_2017_24_bg_500k/cb_2017_24_bg_500k.shp'
block_group_gdf = gpd.read_file(blockgroup_filename)
block_group_gdf = block_group_gdf[(block_group_gdf['STATEFP'] == '24') & (block_group_gdf['COUNTYFP'] == '510')]
block_group_gdf['BLKGRPCE'] = pandas.to_numeric(block_group_gdf['BLKGRPCE'])
block_group_gdf = block_group_gdf.to_crs(tracts_2010_gdf.crs)
e = time.time()
print('Read {0:,.0f} block groups in {1:.3f} seconds.'.format(len(block_group_gdf), e-s))

s = time.time()
zipcodes_filename = shapefile_dir + 'zipcodes/ZIP_Codes.shp'
zipcodes_gdf = gpd.read_file(zipcodes_filename)
zipcodes_gdf = zipcodes_gdf.set_index('ZIPCODE1')
zipcodes_gdf = zipcodes_gdf.to_crs(tracts_2010_gdf.crs)
e = time.time()
print('Read {0:,.0f} zip code shapes in {1:,.3f} seconds.'.format(len(zipcodes_gdf), e-s))

outline_filename = shapefile_dir + 'baltimore_city_polygon/baltimore_city_polygon.shp'
city_outline_gdf = gpd.read_file(outline_filename)
city_outline_gdf = city_outline_gdf.to_crs(tracts_2010_gdf.crs)
print('\nRead city outline...')

water_filename = shapefile_dir + 'water/water.shp'
water_gdf = gpd.read_file(water_filename)
water_gdf = water_gdf.set_index('OBJECTID')
water_gdf = water_gdf.to_crs(tracts_2010_gdf.crs)
print('Read water outlines...')

s = time.time()
streets_filename = shapefile_dir + 'streets/streetcl.shp'
streets_gdf = gpd.read_file(streets_filename)
streets_gdf = streets_gdf.set_index('OBJECTID')
streets_gdf.to_crs(tracts_2010_gdf.crs)
e = time.time()
print('Read {0:,.0f} street centerlines in {1:,.1f} seconds.'.format(len(streets_gdf), e-s))

print('backing up...')
block_group_gdf_bk = block_group_gdf

print('Done!')

In [ ]:
print('restoring block groups file from backup...')
block_group_gdf = block_group_gdf_bk

statnb = pandas.read_csv('/home/idies/workspace/Storage/raddick/persistent/baltimore_neighborhoods/block_groups_to_stat_nbhs.csv', low_memory=False)
statnb['census_tract'] = pandas.to_numeric(statnb['census_tract'], downcast='integer')

statnb['census_tract_str'] = statnb['census_tract'].dropna().apply(lambda x: '{:06d}'.format(int(x*100)))
statnb['nWhite'] = statnb['pct_white'] * statnb['total_pop']
statnb['nBlack'] = statnb['pct_black'] * statnb['total_pop']

block_group_gdf = block_group_gdf.merge(
    statnb, 
    left_on=['TRACTCE', 'BLKGRPCE'], right_on=['census_tract_str', 'block_group'])

print('Organizing by statistical neighborhood...')
stat_nb_df = pandas.DataFrame()

stat_nb_df['neighborhood'] = block_group_gdf['stat_nb'].drop_duplicates()
stat_nb_df = stat_nb_df.set_index('neighborhood')

print('Calculating statistical neighborhood populations...')
nBlockGroups_s = block_group_gdf.groupby('stat_nb').size()
nBlockGroups_s.name = 'nBlockGroups'
popBlockGroups_s = block_group_gdf.groupby('stat_nb').agg({'total_pop': 'sum', 'nWhite': 'sum', 'nBlack': 'sum'})
popBlockGroups_s.name = 'popBlockGroups'

stat_nb_df = stat_nb_df.join(nBlockGroups_s)
stat_nb_df = stat_nb_df.join(popBlockGroups_s)

stat_nb_df['pct_white'] = stat_nb_df['nWhite'] / stat_nb_df['total_pop']
stat_nb_df['pct_black'] = stat_nb_df['nBlack'] / stat_nb_df['total_pop']
stat_nb_df = stat_nb_df.drop(['nWhite', 'nBlack'], axis=1)
stat_nb_df = stat_nb_df.sort_index()

print('Combining geometries...')
geolist = []
for nb in stat_nb_df.index:
    to_unify = block_group_gdf[block_group_gdf['stat_nb'] == nb].geometry.tolist()
    boundary = cascaded_union(to_unify)
    geolist.append(boundary)

stat_nb_df['geometry'] = geolist
stat_nb_gdf = gpd.GeoDataFrame(data=stat_nb_df, geometry='geometry', crs=neighborhood_gdf.crs)

print('backing up...')
stat_nb_gdf_bk = stat_nb_gdf

print('Done!')

## Remove zero-population areas from statistical neighborhood geometries

In [ ]:
print('restoring from backup...')
stat_nb_gdf = stat_nb_gdf_bk

# Mask out the harbor
s = time.time()
print('Masking out water features...')
for snb, thisrow in stat_nb_gdf.iterrows():
    for harborpiece, thatrow in water_gdf[water_gdf['NAME'] == 'Harbor'].iterrows():
        if (stat_nb_gdf.loc[snb].geometry.intersects(thatrow.geometry)):
            stat_nb_gdf.geometry.loc[snb] = stat_nb_gdf.loc[snb].geometry.difference(thatrow.geometry)
        
print('Masking out zero-population areas...')
for snb, thisrow in stat_nb_gdf.iterrows():
    for zeropoparea, thatrow in neighborhood_gdf[neighborhood_gdf['Population'] == 0].iterrows():
        if (stat_nb_gdf.loc[snb].geometry.intersects(thatrow.geometry)):
            stat_nb_gdf.geometry.loc[snb] = stat_nb_gdf.loc[snb].geometry.difference(thatrow.geometry)

print('Masking out low-population areas...')
for snb, thisrow in stat_nb_gdf.iterrows():
    for zeropoparea, thatrow in neighborhood_gdf.loc[['Fairfield Area', 'Pulaski Industrial Area', 'Canton Industrial Area', 'Hawkins Point']].iterrows():
        if (stat_nb_gdf.loc[snb].geometry.intersects(thatrow.geometry)):
            stat_nb_gdf.geometry.loc[snb] = stat_nb_gdf.loc[snb].geometry.difference(thatrow.geometry)

            
print('fixing fractured geometries...')
fixlist = []
for nhb in stat_nb_gdf.index:
    try:
        xx = list(stat_nb_gdf.loc[nhb].geometry)
        if (len(xx) > 1):
            fixlist.append(nhb)
    except TypeError:
        pass

for fixthese in fixlist:
    alist = list(stat_nb_gdf.loc[fixthese].geometry)
    maxarea = 0
    bigoneindex = -1
    for i in range(0,len(alist)):
        if (alist[i].area > maxarea):
            bigoneindex = i
            maxarea = alist[i].area
    bigonegeo = alist[bigoneindex]
    stat_nb_gdf.geometry.loc[fixthese] = bigonegeo


#stat_nb_gdf = stat_nb_gdf[stat_nb_gdf.index != '0']
e = time.time()
print('Done in {0:,.0f} seconds!'.format(e-s))

#stat_nb_gdf.plot()


### Give each neighborhood a unique color for easy identification

In [ ]:
#s = time.time()
#zeropops = len(neighborhood_gdf[neighborhood_gdf['Population'] == 0])
#neighborhood_gdf = neighborhood_gdf[neighborhood_gdf['Population'] > 0]
#e = time.time()
#print('removed {0:,.0f} zero-population neighborhoods in {1:,.3f} seconds.'.format(zeropops, e-s))

stat_nb_gdf['mycolor'] = ''
for nb, thisrow in stat_nb_gdf.iterrows():
    stat_nb_gdf.loc[nb, 'mycolor'] = '#{:06x}'.format(random.randint(0, 0xCCCCCC))
#stat_nb_gdf
print('Done')

# Compare stuff to statistical neighborhoods

In [ ]:
scalefactor = 0.25 # Use 1 for final map

show_tracts = True
show_blockgroups = True
show_csas = True
show_traditional_neighborhoods = False
show_statistical_neighborhoods = True

show_streets = False

fig, ax = plt.subplots(figsize=(48*scalefactor,48*scalefactor))

print('city outline...')
city_outline_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=6*scalefactor, alpha=0.75)

if (show_tracts):
    print('census tracts...')
    tracts_2017_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2*scalefactor, linestyle='-', alpha=0.75)
    for ix, thisrow in tracts_2017_gdf.iterrows():
        annotator = ix
        ax.annotate(annotator, 
                    xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    ha='center', va='center', fontsize=24*scalefactor, alpha=0.5)#, backgroundcolor='white')

if (show_blockgroups):
    print('block groups...')
    block_group_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2.5*scalefactor, linestyle=':', alpha=0.5)
    for ix, thisrow in block_group_gdf.iterrows():
        annotator = thisrow['NAME']
        ax.annotate(annotator, 
                    xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    ha='center', va='center', fontsize=18*scalefactor, color='black', alpha=0.5)

if (show_csas):
    print('community statistical areas...')
    csa_gdf.plot(ax=ax, color='none', edgecolor='green', linewidth=5*scalefactor)
    for ix, thisrow in csa_gdf.iterrows():
        annotator = ix.replace('-', '- ')
        annotator = annotator.replace('/', '/ ')
        annotator = annotator.replace(' ', '\n')
        annotator = annotator.upper()
        ax.annotate(annotator, 
                    xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    ha='center', va='center', fontsize=28*scalefactor, color='green', weight='bold')

if (show_traditional_neighborhoods):
    print('traditional neighborhoods...')
    #neighborhood_gdf.plot(ax=ax, color='none', edgecolor='none', linewidth=0)
    neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=3*scalefactor)
    for ix, thisrow in neighborhood_gdf.iterrows():
        annotator = ix.replace('-', '- ')
        annotator = annotator.replace('/', '/ ')
        annotator = annotator.replace(' ', '\n')
        ax.annotate(annotator, 
                    xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    ha='center', va='center', fontsize=18*scalefactor, color='blue')
                
if (show_statistical_neighborhoods):
    print('statistical neighborhoods...')
    stat_nb_gdf.plot(ax=ax, color=stat_nb_gdf['mycolor'], edgecolor='red', linewidth=3*scalefactor, alpha=0.25)
    for ix, thisrow in stat_nb_gdf.iterrows():
        annotator = ix.replace('-', '- ')
        annotator = annotator.replace('/', '/ ')
        annotator = annotator.replace(' ', '\n')
        ax.annotate(annotator, 
                    xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    ha='center', va='center', fontsize=15*scalefactor, color='red', weight='bold')

if (show_streets):
    print('streets...')
    streets_gdf.plot(ax=ax, color='black', linewidth=1*scalefactor, alpha=0.10)

print('water...')
water_gdf.plot(ax=ax, color='cyan')

print('hiding axis labels...')
#ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)

z_gdf.plot(marker='*', color='black', markersize=35, ax=ax)
plt.title('Dividing Baltimore into {0:.0f} neighborhoods from census block groups'.format(len(stat_nb_gdf)), fontsize=64*scalefactor, y=1.02)

#plt.savefig('baltimore_neighborhoods_final.svg', format='svg')
#print('OK')

plt.show()
#g_gdf

### Output data on each statistical neighborhood

<ul>
    <li>Area in acres</li>
    <li>Total population</li>
    <li>Percent white residents</li>
    <li>Percent black residents</li>
    <li>(note: those percentages won't quite add up to 100%)</li>
</ul>





In [ ]:
#stat_nb_gdf.geometry.apply(lambda x: x.area())
thepath = '/home/idies/workspace/Storage/raddick/persistent/baltimore_neighborhoods/'
thefile = thepath + 'neighborhood_data.csv'

tost = stat_nb_gdf.copy()
tost = tost.to_crs({'init': 'epsg:3857'})
#tost.head(2)
area_acres = tost['geometry'].area / 4046.86 # convert to acres
stat_nb_gdf['area_acres'] = area_acres
stat_nb_gdf[['nBlockGroups', 'area_acres', 'total_pop', 'pct_white', 'pct_black']].to_csv(thefile)
print('wrote data')



In [ ]:
#import geocoder
#gpd.geocode.geocode("3400 North Charles Street, Baltimore, MD, 21218")
g = geocoder.bing('Mountain View, CA', key='Agrc_VFxa6iK3mVYNIC1Mcao2TwVTPG5tDbok7UbDcCYf5PRGmnaeLF_Wm_znHeo')
#g.latlng
#print('ok')
#import requests
#url = 'https://maps.googleapis.com/maps/api/geocode/json'
#params = {'sensor': 'false', 'address': 'Mountain View, CA'}
#r = requests.get(url, params=params)
#results = r.json()['results']
#location = results[0]['geometry']['location']
#location['lat'], location['lng']
g.latlng
thegeometry = Point(g.latlng)

g_df = pandas.DataFrame(data=g.latlng)#gpd.GeoDataFrame(data=g.latlng, crs=stat_nb_gdf.crs)#, geometry=thegeometry)
#
g_df = g_df.T
g_df = g_df.rename(columns={0: 'lat', 1: 'long'})
g_df = g_df.assign(geometry=Point(g_df['lat'], g_df['long']))
g_df
g_gdf = gpd.GeoDataFrame(data=g_df, crs=neighborhood_gdf.crs, geometry='geometry')
g_gdf = g_gdf.set_geometry('geometry')

z_gdf = g_gdf.to_crs(neighborhood_gdf.crs)


#z_gdf = g_gdf.set_geometry('geometry').to_crs(neighborhood_gdf.crs)
#z_gdf
world = world.to_crs({'init': 'epsg:3395'})
cities = cities.to_crs(world.crs)
base = world.plot(color='white', edgecolor='black')
z_gdf = z_gdf.to_crs(world.crs)
z_gdf.plot(ax=base, marker='o', color='red', markersize=5)
#base = stat_nb_gdf.plot()
#g_gdf.plot(ax=base, marker='o', color='red', markersize=5)

plt.show()
#print('ok')

In [ ]:

#import geocoder
businessaddress = '3400 North Charles St. Baltimore, MD 21218'
g = geocoder.bing(businessaddress, key='Agrc_VFxa6iK3mVYNIC1Mcao2TwVTPG5tDbok7UbDcCYf5PRGmnaeLF_Wm_znHeo')
thegeometry = Point(g.latlng)
g_df = pandas.DataFrame(data=g.latlng)
g_df = g_df.T
g_df = g_df.rename(columns={0: 'lat', 1: 'long'})
g_df = g_df.assign(geometry=Point(g_df['long'], g_df['lat']))
g_gdf = gpd.GeoDataFrame(data=g_df, geometry='geometry')
#world = world.to_crs({'init': 'epsg:3395'})
#cities = cities.to_crs(world.crs)
base = tracts_2017_gdf.plot()
#base = stat_nb_gdf.plot()
g_gdf.plot(ax=base, marker='o', color='red', markersize=5)
plt.show()
#g_gdf
#tracts_2017_gdf.head(1)

# Compare block groups, census tracts, and community statistical areas

In [ ]:
fig, ax = plt.subplots(figsize=(38,38))

block_group_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle=':', alpha=0.5)

tracts_2017_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle='-', alpha=0.75)

for ix, thisrow in block_group_gdf.iterrows():
    annotator = thisrow['NAME']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=11, color='black', alpha=0.5)

for ix, thisrow in tracts_2017_gdf.iterrows():
    annotator = ix
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=18, alpha=0.75)

csa_gdf.plot(ax=ax, color='none', edgecolor='green', linewidth=4)

for ix, thisrow in csa_gdf.iterrows():
    annotator = ix.replace('-', '- ')
    annotator = annotator.replace('/', '/ ')
    annotator = annotator.replace(' ', '\n')
    annotator = annotator.upper()
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=24, color='green')

#neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=2)

#water_gdf[water_gdf['NAME'] == 'Harbor'].plot(ax=ax, color='blue')

ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)

#print('saving...')
#plt.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/baltimore_boundary_guide_csa.svg', format='svg')
#print('ok')
plt.show()
#csa_gdf

# Compare block groups, census tracts, and neighborhoods

In [ ]:
fig, ax = plt.subplots(figsize=(38,38))

block_group_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle=':', alpha=0.5)

tracts_2017_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle='-')


for ix, thisrow in block_group_gdf.iterrows():
    annotator = thisrow['NAME']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=9, color='black', alpha=0.5)

for ix, thisrow in tracts_2017_gdf.iterrows():
    annotator = ix
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=12)
    
csa_gdf.plot(ax=ax, color='none', edgecolor='green', linewidth=4)

#for ix, thisrow in csa_gdf.iterrows():
#    annotator = ix.replace('-', '- ')
#    annotator = annotator.replace('/', '/ ')
#    annotator = annotator.replace(' ', '\n')
#    annotator = annotator.upper()
#    ax.annotate(annotator, 
#                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
#                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
#                ha='center', va='center', fontsize=24, color='green')

neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=2)

for ix, thisrow in neighborhood_gdf.iterrows():
    annotator = ix.replace('-', '- ')
    annotator = annotator.replace('/', '/ ')
    annotator = annotator.replace(' ', '\n')
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=14, color='red')

#water_gdf.plot(ax=ax, color='blue')
water_gdf[water_gdf['NAME'] == 'Harbor'].plot(ax=ax, color='blue')

ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)

plt.show()
#csa_gdf
#plt.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/baltimore_boundary_guide.svg', format='svg')
print('ok')

In [ ]:
cols = neighborhood_gdf.columns.tolist()
cols.remove('created_us')
cols.remove('created_da')
cols.remove('last_edite')
cols.remove('last_edi_1')
cols.remove('ShapeSTAre')
cols.remove('ShapeSTLen')
cols.remove('geometry')
#os.getcwd()
neighborhood_gdf[cols].to_csv('neighborhood_full_data.csv', encoding='utf-8')
print('ok')

In [ ]:
fig, ax = plt.subplots(figsize=(38,38))

block_group_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle=':', alpha=0.5)
tracts_2017_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle='-', alpha=0.75)

for ix, thisrow in block_group_gdf.iterrows():
    annotator = thisrow['NAME']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=11, color='black', alpha=0.5)

for ix, thisrow in tracts_2017_gdf.iterrows():
    annotator = ix
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=18, alpha=0.75)

neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=2)

ax.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')
plt.show()

# Compare zip codes and neighborhoods

In [ ]:
fig, ax = plt.subplots(figsize=(38,38))

neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=2)

for ix, thisrow in neighborhood_gdf.iterrows():
    annotator = ix.replace('-', '- ')
    annotator = annotator.replace('/', '/ ')
    annotator = annotator.replace(' ', '\n')
#    print(thisrow.geometry.centroid.x)
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=14, color='red')

zipcodes_gdf.plot(ax=ax, color='none', edgecolor='blue', linewidth=5)
#print('------------------------\n\n')
for ix, thisrow in zipcodes_gdf.iterrows():
    annotator = ix
#    print(annotator)
#    print(thisrow.geometry.centroid.x)
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=36, color='blue')

ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)
plt.show()

In [ ]:
neighborhood_gdf[neighborhood_gdf.index.map(lambda x: x[0] == 'W')]
#block_group_gdf[block_group_gdf['TRACTCE'] == '010400']
#print('ok')

In [ ]:
#streets_gdf.sample(2).T
streets_gdf.groupby('SUBTYPE').size()

In [ ]:
# STREX: expresswway
# STRR: ramp
# STRTN: tunnel
# STRPRD: primary road
# STRALY: alley
fig, ax = plt.subplots(figsize=(15,15))
streets_gdf[streets_gdf['SUBTYPE'].apply(lambda x: x in ['STREX'])].plot(ax=ax, color='red')
streets_gdf[streets_gdf['SUBTYPE'].apply(lambda x: x in ['STRPRD'])].plot(ax=ax, color='blue')
plt.show()

In [ ]:
block_group_gdf

In [ ]:
statnb['total_pop']

#statnb['total_pop'] = pandas.to_numeric(statnb['total_pop'], errors='coerce')
statnb.dtypes

In [ ]:
block_group_gdf

## Sort through one tract at a time to identify neighborhoods

In [ ]:
thistract = 2805
thistractstr = '280500'
nhoods = ['Penn-Fallsway', 'Jonestown', 'Oldtown', 'Pleasant View Gardens']
show_streets = True
show_water = False

fig, ax = plt.subplots(figsize=(15,15))

z = tracts_2010_gdf[tracts_2010_gdf.index == thistract]
#z.plot(ax=ax, color='none', edgecolor='black', linewidth=8, linestyle='-', alpha=0.75)

for ix, thisrow in z.iterrows():
    annotator = ix
#    ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', fontsize=18, alpha=1)

zb = block_group_gdf[block_group_gdf['TRACTCE'] == thistractstr]
zb = zb[zb['BLKGRPCE'] == '2']
zb.plot(ax=ax, color='none', edgecolor='black', linewidth=8, linestyle=':')
for ix, thisrow in zb.iterrows():
    annotator = thisrow['BLKGRPCE']
    ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', fontsize=18, alpha=1)

zz = neighborhood_gdf[neighborhood_gdf.index.map(lambda x: x in nhoods)]
zz.plot(ax=ax, color='none', edgecolor='red', linewidth=8, linestyle='-')
for ix, thisrow in zz.iterrows():
    annotator = ix
    ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', color='red', fontsize=18, alpha=0.75)

grapharea_s = gpd.GeoSeries(data=[
    Polygon([
        (plt.xlim()[0],plt.ylim()[0]), 
        (plt.xlim()[1],plt.ylim()[0]), 
        (plt.xlim()[1],plt.ylim()[1]), 
        (plt.xlim()[0],plt.ylim()[1])
    ])
], crs=neighborhood_gdf.crs)

grapharea_gdf = gpd.GeoDataFrame(data=None, geometry=grapharea_s.geometry, crs=neighborhood_gdf.crs)

if (show_streets):
    sy_gdf = streets_gdf[
        (streets_gdf['SUBTYPE'].apply(lambda x: x in ['STRPRD']))
    ]
    sy_gdf = sy_gdf[sy_gdf.geometry.within(grapharea_gdf.loc[0].geometry)]
    sy_gdf.geometry.plot(ax=ax, linewidth=1, color='purple')
    for ix, thisrow in sy_gdf.iterrows():
        annotator = thisrow['FULLNAME']
        ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', color='purple', 
                    fontsize=8)
if (show_water):
    ww_gdf = water_gdf[water_gdf.geometry.within(grapharea_gdf.loc[0].geometry)]
    ww_gdf.geometry.plot(ax=ax, linewidth=1, color='cyan')
    for ix, thisrow in ww_gdf.iterrows():
        annotator = thisrow['NAME']
        ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', color='blue', fontsize=8)
        
#ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)
grapharea_gdf.plot(ax=ax, color='none', edgecolor='orange', linewidth=6)

plt.show()